TSG094 - Grafana logs
=====================

Steps
-----

### Parameters

In [ ]:
import re

tail_lines = 2000

pod = None # All
container = "grafana"
log_files = [ "/var/log/supervisor/log/grafana*.log" ]

expressions_to_analyze = []

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os

try:
    from kubernetes import client, config
    from kubernetes.stream import stream

    if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
        config.load_incluster_config()
    else:
        try:
            config.load_kube_config()
        except:
            display(Markdown(f'HINT: Use [TSG112 - App-Deploy Proxy Nginx Logs](../log-analyzers/tsg112-get-approxy-nginx-logs.ipynb) to resolve this issue.'))
            raise
    api = client.CoreV1Api()

    print('Kubernetes client instantiated')
except ImportError:
    from IPython.display import Markdown
    display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
    raise

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Get tail for log

In [ ]:
# Display the last 'tail_lines' of files in 'log_files' list

pods = api.list_namespaced_pod(namespace)

entries_for_analysis = []

for p in pods.items:
    if pod is None or p.metadata.name == pod:
        for c in p.spec.containers:
            if container is None or c.name == container:
                for log_file in log_files:
                    print (f"- LOGS: '{log_file}' for CONTAINER: '{c.name}' in POD: '{p.metadata.name}'")
                    try:
                        output = stream(api.connect_get_namespaced_pod_exec, p.metadata.name, namespace, command=['/bin/sh', '-c', f'tail -n {tail_lines} {log_file}'], container=c.name, stderr=True, stdout=True)
                    except Exception:
                        print (f"FAILED to get LOGS for CONTAINER: {c.name} in POD: {p.metadata.name}")
                    else:
                        for line in output.split('\n'):
                            for expression in expressions_to_analyze:
                                if expression.match(line):
                                    entries_for_analysis.append(line)
                            print(line)
print("")
print(f"{len(entries_for_analysis)} log entries found for further analysis.")

### Analyze log entries and suggest relevant Troubleshooting Guides

In [ ]:
# Analyze log entries and suggest further relevant troubleshooting guides

from IPython.display import Markdown

import os
import json
import requests
import ipykernel
import datetime

from urllib.parse import urljoin
from notebook import notebookapp

def get_notebook_name():
    """Return the full path of the jupyter notebook.   Some runtimes (e.g. ADS) 
    have the kernel_id in the filename of the connection file.  If so, the 
    notebook name at runtime can be determined using `list_running_servers`.
    Other runtimes (e.g. azdata) do not have the kernel_id in the filename of
    the connection file, therefore we are unable to establish the filename
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    
    # If the runtime has the kernel_id in the connection filename, use it to
    # get the real notebook name at runtime, otherwise, use the notebook 
    # filename from build time.
    try: 
        kernel_id = connection_file.split('-', 1)[1].split('.')[0]
    except:
        pass
    else:
        for servers in list(notebookapp.list_running_servers()):
            try:
                response = requests.get(urljoin(servers['url'], 'api/sessions'), params={'token': servers.get('token', '')}, timeout=.01)
            except:
                pass
            else:
                for nn in json.loads(response.text):
                    if nn['kernel']['id'] == kernel_id:
                        return nn['path']

def load_json(filename):
    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def get_notebook_rules():
    """Load the notebook rules from the metadata of this notebook (in the .ipynb file)"""
    file_name = get_notebook_name()

    if file_name == None:
        return None
    else:
        j = load_json(file_name)

        if "azdata" not in j["metadata"] or \
            "expert" not in j["metadata"]["azdata"] or \
            "log_analyzer_rules" not in j["metadata"]["azdata"]["expert"]:
            return []
        else:
            return j["metadata"]["azdata"]["expert"]["log_analyzer_rules"]

rules = get_notebook_rules()

if rules == None:
    print("")
    print(f"Log Analysis only available when run in Azure Data Studio.  Not available when run in azdata.")
else:
    hints = 0
    if len(rules) > 0:
        for entry in entries_for_analysis:
            for rule in rules:
                if entry.find(rule[0]) != -1:
                    print (entry)

                    display(Markdown(f'HINT: Use [{rule[2]}]({rule[3]}) to resolve this issue.'))
                    hints = hints + 1

    print("")
    print(f"{len(entries_for_analysis)} log entries analyzed (using {len(rules)} rules). {hints} further troubleshooting hints made inline.")

In [ ]:
print('Notebook execution complete.')